In [16]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import SVDpp
from surprise import SlopeOne
from surprise import accuracy
from surprise.model_selection import GridSearchCV

## Data Loading

In [2]:
df = pd.read_json('Digital_Music_5.json', lines=True)
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,3.0,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,{'Format:': ' Audio CD'},Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,1370217600,NaN
1,5,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,{'Format:': ' Audio CD'},Ad,bien,Five Stars,1412985600,NaN
2,5,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,{'Format:': ' Audio CD'},JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,NaN
3,4,3.0,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,1386374400,NaN
4,5,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,{'Format:': ' Audio CD'},melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",1465689600,NaN


In [3]:
df['unixReviewTime'] = df['unixReviewTime'].map(lambda x: datetime.fromtimestamp(x))

In [4]:
df['style'] = df['style'].astype(str)
df['style'] = df['style'].apply(lambda x: x.split(':')[-1].strip(" }'"))

In [5]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,3.0,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,Audio CD,Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,2013-06-02 17:00:00,NaN
1,5,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,Audio CD,Ad,bien,Five Stars,2014-10-10 17:00:00,NaN
2,5,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,Audio CD,JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,2014-02-10 16:00:00,NaN
3,4,3.0,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,Audio CD,john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,2013-12-06 16:00:00,NaN
4,5,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,Audio CD,melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",2016-06-11 17:00:00,NaN


## KNN Basic Model

In [12]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.02, 0.04, 0.06]}

gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3)

gs.fit(data)


best_params = gs.best_params['rmse']

print("Best Parameters:", best_params)

best_knn_basic_model = KNNBasic(n_epochs=best_params['n_epochs'], 
                     lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])

best_knn_basic_model.fit(data.build_full_trainset())

predictions = best_knn_basic_model.test(testset)

rmse = accuracy.rmse(predictions)
print("RMSE with Tuned Model:", rmse)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

## SVD++ Model

In [15]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.02, 0.04, 0.06]}

gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3)

gs.fit(data)


best_params = gs.best_params['rmse']

print("Best Parameters:", best_params)

best_svd_pp_model = SVDpp(n_epochs=best_params['n_epochs'], 
                     lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])

best_svd_pp_model.fit(data.build_full_trainset())

predictions = best_svd_pp_model.test(testset)

rmse = accuracy.rmse(predictions)
print("RMSE with Tuned Model:", rmse)

Best Parameters: {'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.02}
RMSE: 0.4015
RMSE with Tuned Model: 0.4015394721076082


## Slope One Model

In [18]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

slope_one_model = SlopeOne()

slope_one_model.fit(data.build_full_trainset())

predictions = slope_one_model.test(testset)

rmse = accuracy.rmse(predictions)
print("RMSE with Tuned Model:", rmse)

RMSE: 0.1230
RMSE with Tuned Model: 0.1229537349211004
